## Betti numbers calculation script doc

### Introduction

> This script implements an incremental algorithm to calculate the Betti numbers of a simplicial complex using the boundary matrix and boundary matrix reduction. The Betti numbers indicate the number of independent cycles at each dimension of a topological space.

### Key Concepts
- **Boundary matrix**: This matrix represents the relationships between simplices and their faces.
- **Matrix reduction**: This step simplifies the boundary matrix using column operations to determine independent cycles.

## Functions

### `build_boundary_matrix(simplicial_complex)`

> This function generates the boundary matrix from a given simplicial complex. It takes as input a simplicial complex represented as a list of simplices (e.g., vertices, edges, triangles) and returns the boundary matrix along with the list of all simplices.

### `reduce_boundary_matrix(boundary_matrix)`

> This function reduces the boundary matrix using column operations. It returns the reduced matrix and a list of signs associated with each column (positive or negative), indicating whether the column corresponds to an independent cycle.

### `incremental_algorithm(simplicial_complex)`

> This function applies the incremental algorithm to compute the Betti numbers. It uses `build_boundary_matrix` and `reduce_boundary_matrix` functions to generate the necessary matrices and determine the Betti numbers based on independent cycles in the complex.

## Example Execution

### 1. Input as list

> For the list, we use the simplicial complex as follows:
> 
>> Circle: [[0], [1], [2], [0, 1], [1, 2], [2, 0]]
>>
>> Sphere: [[0], [1], [2], [3], [0, 1], [1, 2], [2, 3], [3, 0], [1, 3], [0, 2], [0, 1, 2], [1, 2, 3], [2, 3, 0], [3, 0, 1]]
>>
>> Torus: [[0], [1], [2], [3], [4], [5], [6], [7], [8], [0, 1], [1, 2], [2, 0], [0, 3], [3, 1], [1, 5], [5, 2], [2, 6], [6, 0], [3, 5], [5, 6], [6, 3], [3, 4], [4, 5], [5, 7], [7, 6], [6, 8], [8, 3], [4, 7], [7, 8], [8, 4], [4, 0], [0, 7], [7, 1], [1, 8], [8, 2], [2, 4], [0, 1, 3], [1, 3, 5], [1, 2, 5], [2, 5, 6], [2, 0, 6], [0, 6, 3], [3, 5, 4], [5, 4, 7], [5, 6, 7], [6, 7, 8], [6, 3, 8], [3, 8, 4], [4, 7, 0], [7, 0, 1], [7, 8, 1], [8, 1, 2], [8, 4, 2], [4, 2, 0]]

> ### Betti numbers results

>> Circle: `[1, 1]`
>> 
>> Sphere: `[1, 0, 1]`
>> 
>> Torus: `[1, 2, 1]`

In [62]:
import numpy as np
from itertools import combinations

def build_boundary_matrix(sc):
    all_splex = sorted([tuple(sorted(s)) for s in sc], key=lambda s: (len(s), s))
    splex_indices = {splex: idx for idx, splex in enumerate(all_splex)}

    n = len(all_splex)
    boundary_matrix = np.zeros((n, n), dtype=int)

    for s in sc:
        col = splex_indices[tuple(sorted(s))]
        for face in combinations(s, len(s) - 1):
            face_sorted = tuple(sorted(face))
            if face_sorted in splex_indices:
                row = splex_indices[face_sorted]
                boundary_matrix[row, col] = 1

    return boundary_matrix, all_splex

def reduce_boundary_matrix(boundary_matrix):
    rows, cols = boundary_matrix.shape
    reduced_matrix = boundary_matrix.copy()
    sigma_signs = ['positive'] * cols

    for j in range(cols):
        pivot_row = -1
        for i in range(rows - 1, -1, -1):
            if reduced_matrix[i, j] == 1:
                pivot_row = i
                break

        if pivot_row == -1:
            sigma_signs[j] = 'positive'
            continue

        sigma_signs[j] = 'negative'

        for k in range(j + 1, cols):
            if reduced_matrix[pivot_row, k] == 1:
                reduced_matrix[:, k] = (reduced_matrix[:, k] + reduced_matrix[:, j]) % 2

    return reduced_matrix, sigma_signs

def incremental_algorithm(sc):
    boundary_matrix, all_splex = build_boundary_matrix(sc)
    reduced_matrix, sigma_signs = reduce_boundary_matrix(boundary_matrix)

    betti_numbers = [0] * (max(len(s) for s in sc) + 1)

    for j, splex in enumerate(all_splex):
        dim = len(splex) - 1
        if sigma_signs[j] == 'positive':
            betti_numbers[dim] += 1
        else:
            if dim > 0:
                betti_numbers[dim - 1] -= 1

    while len(betti_numbers) > 1 and betti_numbers[-1] == 0:
        betti_numbers.pop()

    return betti_numbers

def example_test():
    circle = [[0], [1], [2], [0, 1], [1, 2], [2, 0]]
    sphere = [[0], [1], [2], [3], [0, 1], [1, 2], [2, 3], [3, 0], [1, 3], [0, 2], [0, 1, 2], [1, 2, 3], [2, 3, 0], [3, 0, 1]]
    torus = [[0], [1], [2], [3], [4], [5], [6], [7], [8], [0, 1], [1, 2], [2, 0], [0, 3], [3, 1], [1, 5], [5, 2], [2, 6], [6, 0], [3, 5], [5, 6], [6, 3], [3, 4], [4, 5], [5, 7], [7, 6], [6, 8], [8, 3], [4, 7], [7, 8], [8, 4], [4, 0], [0, 7], [7, 1], [1, 8], [8, 2], [2, 4], [0, 1, 3], [1, 3, 5], [1, 2, 5], [2, 5, 6], [2, 0, 6], [0, 6, 3], [3, 5, 4], [5, 4, 7], [5, 6, 7], [6, 7, 8], [6, 3, 8], [3, 8, 4], [4, 7, 0], [7, 0, 1], [7, 8, 1], [8, 1, 2], [8, 4, 2], [4, 2, 0]]

    print("Betti numbers for a circle:", incremental_algorithm(circle))
    print("Betti numbers for a sphere:", incremental_algorithm(sphere))
    print("Betti numbers for a torus:", incremental_algorithm(torus))

example_test()

Betti numbers for a circle: [1, 1]
Betti numbers for a sphere: [1, 0, 1]
Betti numbers for a torus: [1, 2, 1]


## 2. For input as dictionnary

> For the dictionnary format, we can use:
> 
> >circle = {0: [[0], [1], [2]], 1: [[0, 1], [1, 2], [2, 0]]}
> >
> >sphere = {0: [[0], [1], [2], [3]], 1: [[0, 1], [1, 2], [2, 3], [3, 0]], 2: [[0, 1, 2], [1, 2, 3], [2, 3, 0], [3, 0, 1]]}
> >
> >torus = {0: [[0], [1], [2], [3], [4], [5], [6], [7], [8]],
             1: [[0, 1], [1, 2], [2, 0], [0, 3], [3, 1], [1, 5], [5, 2], [2, 6], [6, 0], [3, 5], [5, 6], [6, 3], [3, 4], [4, 5], [5, 7], [7, 6], [6, 8], [8, 3], [4, 7], [7, 8], [8, 4], [4, 0], [0, 7], [7, 1], [1, 8], [8, 2], [2, 4]],
             2: [[0, 1, 3], [1, 3, 5], [1, 2, 5], [2, 5, 6], [2, 0, 6], [0, 6, 3], [3, 5, 4], [5, 4, 7], [5, 6, 7], [6, 7, 8], [6, 3, 8], [3, 8, 4], [4, 7, 0], [7, 0, 1], [7, 8, 1], [8, 1, 2], [8, 4, 2], [4, 2, 0]]}


In [63]:
def build_boundary_matrix(sc_dict):
    all_splex = sorted([tuple(sorted(s)) for s in sum(sc_dict.values(), [])], key=lambda s: (len(s), s))
    splex_indices = {splex: idx for idx, splex in enumerate(all_splex)}

    n = len(all_splex)
    boundary_matrix = np.zeros((n, n), dtype=int)

    for key, simplices in sc_dict.items():
        for s in simplices:
            col = splex_indices[tuple(sorted(s))]
            for face in combinations(s, len(s) - 1):
                face_sorted = tuple(sorted(face))
                if face_sorted in splex_indices:
                    row = splex_indices[face_sorted]
                    boundary_matrix[row, col] = 1

    return boundary_matrix, all_splex

def reduce_boundary_matrix(boundary_matrix):
    rows, cols = boundary_matrix.shape
    reduced_matrix = boundary_matrix.copy()
    sigma_signs = ['positive'] * cols

    for j in range(cols):
        pivot_row = -1
        for i in range(rows - 1, -1, -1):
            if reduced_matrix[i, j] == 1:
                pivot_row = i
                break

        if pivot_row == -1:
            sigma_signs[j] = 'positive'
            continue

        sigma_signs[j] = 'negative'

        for k in range(j + 1, cols):
            if reduced_matrix[pivot_row, k] == 1:
                reduced_matrix[:, k] = (reduced_matrix[:, k] + reduced_matrix[:, j]) % 2

    return reduced_matrix, sigma_signs

def incremental_algorithm(sc_dict):
    boundary_matrix, all_splex = build_boundary_matrix(sc_dict)
    reduced_matrix, sigma_signs = reduce_boundary_matrix(boundary_matrix)

    max_dim = max(len(s) for s in sum(sc_dict.values(), [])) - 1
    betti_numbers = [0] * (max_dim + 1)

    for j, splex in enumerate(all_splex):
        dim = len(splex) - 1
        if sigma_signs[j] == 'positive':
            betti_numbers[dim] += 1
        else:
            if dim > 0:
                betti_numbers[dim - 1] = max(0, betti_numbers[dim - 1] - 1)

    while len(betti_numbers) > 1 and betti_numbers[-1] == 0:
        betti_numbers.pop()

    return betti_numbers


def example_test():
    circle = {0: [[0], [1], [2]], 1: [[0, 1], [1, 2], [2, 0]]}
    sphere = {0: [[0], [1], [2], [3]], 1: [[0, 1], [1, 2], [2, 3], [3, 0]], 2: [[0, 1, 2], [1, 2, 3], [2, 3, 0], [3, 0, 1]]}
    torus = {0: [[0], [1], [2], [3], [4], [5], [6], [7], [8]],
             1: [[0, 1], [1, 2], [2, 0], [0, 3], [3, 1], [1, 5], [5, 2], [2, 6], [6, 0], [3, 5], [5, 6], [6, 3], [3, 4], [4, 5], [5, 7], [7, 6], [6, 8], [8, 3], [4, 7], [7, 8], [8, 4], [4, 0], [0, 7], [7, 1], [1, 8], [8, 2], [2, 4]],
             2: [[0, 1, 3], [1, 3, 5], [1, 2, 5], [2, 5, 6], [2, 0, 6], [0, 6, 3], [3, 5, 4], [5, 4, 7], [5, 6, 7], [6, 7, 8], [6, 3, 8], [3, 8, 4], [4, 7, 0], [7, 0, 1], [7, 8, 1], [8, 1, 2], [8, 4, 2], [4, 2, 0]]}

    print("Betti numbers for a circle:", incremental_algorithm(circle))
    print("Betti numbers for a sphere:", incremental_algorithm(sphere))
    print("Betti numbers for a torus:", incremental_algorithm(torus))

example_test()

Betti numbers for a circle: [1, 1]
Betti numbers for a sphere: [1, 0, 1]
Betti numbers for a torus: [1, 2, 1]
